In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(
                f"Train Epoch: {epoch} [{(batch_idx * len(data)) // len(train_loader.dataset)}/{len(train_loader.dataset)}] Loss: {loss.item():.4f}] ")


In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # Move both data and target to the same device
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # Sum up the batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}"
          f" ({accuracy:.2f}%)\n")

In [5]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

torch.manual_seed(143421)
if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset1 = datasets.MNIST('data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_utils = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_utils)
    scheduler.step()


Train Epoch: 1 [0/60000] Loss: 2.2919] 
Train Epoch: 1 [0/60000] Loss: 0.9469] 
Train Epoch: 1 [0/60000] Loss: 0.9414] 
Train Epoch: 1 [0/60000] Loss: 0.5574] 
Train Epoch: 1 [0/60000] Loss: 0.4291] 
Train Epoch: 1 [0/60000] Loss: 0.3142] 
Train Epoch: 1 [0/60000] Loss: 0.1761] 
Train Epoch: 1 [0/60000] Loss: 0.2418] 
Train Epoch: 1 [0/60000] Loss: 0.3467] 
Train Epoch: 1 [0/60000] Loss: 0.2373] 
Train Epoch: 1 [0/60000] Loss: 0.1097] 
Train Epoch: 1 [0/60000] Loss: 0.2243] 
Train Epoch: 1 [0/60000] Loss: 0.2363] 
Train Epoch: 1 [0/60000] Loss: 0.1416] 
Train Epoch: 1 [0/60000] Loss: 0.1637] 
Train Epoch: 1 [0/60000] Loss: 0.0997] 
Train Epoch: 1 [0/60000] Loss: 0.2796] 
Train Epoch: 1 [0/60000] Loss: 0.0691] 
Train Epoch: 1 [0/60000] Loss: 0.1335] 
Train Epoch: 1 [0/60000] Loss: 0.2519] 
Train Epoch: 1 [0/60000] Loss: 0.1840] 
Train Epoch: 1 [0/60000] Loss: 0.1415] 
Train Epoch: 1 [0/60000] Loss: 0.2133] 
Train Epoch: 1 [0/60000] Loss: 0.2359] 
Train Epoch: 1 [0/60000] Loss: 0.1317] 


In [6]:
torch.save(model.state_dict(), 'mnist_cnn.pt')